# Bayesian inference with PyMC
## Truncated normal distribution for the likelihood

In [ ]:
import pymc3 as pm

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
np.random.seed(123)

import scipy.stats as st
import scipy.special as ss

import theano.tensor as tt

In [ ]:
def normal_pdf(x, mu, sigma):
    return np.exp(-((x-mu)**2)/(2*sigma**2)) / np.sqrt(2*np.pi*sigma**2)

def normal_cdf(x, mu, sigma):
    return 0.5 * (1 + ss.erf((x-mu)/(sigma*np.sqrt(2))))

In [ ]:
# The following functions are based on Wikipedia. 
# The drawback is that are not easily adaptable to other distributions
def normal_pdf_truncated(x, mu, sigma, upper):
    pdf = normal_pdf(x=(x-mu)/sigma, mu=0, sigma=1)
    Z = normal_cdf(x=(upper-mu)/sigma, mu=0, sigma=1)
    pdf /= (sigma * Z)
    
    return pdf

def normal_cdf_truncated(x, mu, sigma, upper):
    cdf = normal_cdf(x=(x-mu)/sigma, mu=0, sigma=1)    
    Z = normal_cdf(x=(upper-mu)/sigma, mu=0, sigma=1)
    cdf /= Z
    
    return cdf

In [ ]:
# The following functions are based on the general approach to truncate distributions. 
# These can be easily adapted to other distributions
def normal_pdf_truncated_v2(x, mu, sigma, upper):
    pdf = normal_pdf(x=x, mu=mu, sigma=sigma)
    pdf[x>upper] = 0
    
    pdf /= normal_cdf(x=upper, mu=mu, sigma=sigma)
    
    return pdf

def normal_cdf_truncated_v2(x, mu, sigma, upper):
    cdf = normal_cdf(x=x, mu=mu, sigma=sigma)
    cdf[x>upper] = 0
    
    cdf /= normal_cdf(x=upper, mu=mu, sigma=sigma)
    
    return cdf

In [ ]:
# generate Inverse Gaussian observations
mu = 0
sigma = 3

y_norm = np.random.normal(loc=mu, scale=sigma, size=10000)

range_x = 0.01
x_norm = np.arange(np.min(y_norm), np.max(y_norm), range_x)

upper = 5

In [ ]:
# Test if the function are working correctly
plt.hist(y_norm[y_norm<=upper], 
         bins=100, density=True, histtype='stepfilled', facecolor='lightgray', edgecolor='k',
         label='observations')
plt.plot(x_norm[x_norm<=upper], normal_pdf_truncated(x=x_norm[x_norm<=upper], mu=mu, sigma=sigma, upper=upper), 
         'b-', alpha=0.2, lw=7, label='$Truncated_{wikipedia}$');
plt.plot(x_norm, normal_pdf_truncated_v2(x=x_norm, mu=mu, sigma=sigma, upper=upper), 
         color='lightgreen', alpha=1, lw=2, label='$Trucanted_{general}$');

plt.legend();

In [ ]:
# Test the CDF
plt.hist(y_norm[y_norm<=upper], 
         bins=100, density=True, cumulative=True, histtype='stepfilled', facecolor='lightgray', edgecolor='k',
         label='observations');
plt.plot(x_norm[x_norm<=upper], normal_cdf_truncated(x=x_norm[x_norm<=upper], mu=mu, sigma=sigma, upper=upper), 
         'b-', alpha=0.5, lw=5,
         label='$Truncated_{wikipedia}$');
plt.plot(x_norm, normal_cdf_truncated_v2(x=x_norm, mu=mu, sigma=sigma, upper=upper),
         color='lightgreen', alpha=1, lw=2,
         label='$Trucanted_{general}$');
plt.legend();

## Estimate parameter with PyMC3

In [ ]:
obs = y_norm[y_norm<=upper]
obs

In [ ]:
def normal_logcdf_theano(x, mu, sigma):
    return tt.log(0.5 * (1 + tt.erf((x-mu)/(sigma*tt.sqrt(2)))))


In [ ]:
def normal_lcdf(x, mu, sigma):
    z = (x - mu) / sigma
    return tt.switch(
        tt.lt(z, -1.0),
        tt.log(tt.erfcx(-z / tt.sqrt(2.)) / 2.) - tt.sqr(z) / 2.,
        tt.log1p(-tt.erfc(z / tt.sqrt(2.)) / 2.)
    )

In [ ]:
with pm.Model() as model:
    
    mu = pm.Normal('mu', mu=0, sd=100)
    sd = pm.HalfNormal('sd', sd=100)
    
    y_likelihood = pm.Normal('y_likelihood', mu, sd, observed=obs)
    # potential works the same as target +=... in Stan, tt.switch works the same as ifelse
    # see: https://discourse.pymc.io/t/truncated-inverse-normal-distribution-also-known-as-wald-distribution/1125
    upper_bound = pm.Potential('upper_bound', tt.switch(tt.gt(obs, upper), -np.inf, -normal_logcdf_theano(upper, mu, sd)))
    #upper_bound = pm.Potential('upper_bound', tt.switch(tt.gt(obs, upper), -np.inf, -normal_lcdf(upper, mu, sd)))
    
    trace = pm.sample(draws=5000, chains=2, cores=2, tune=1000)

In [ ]:
pm.traceplot(trace, figsize=(5, 4));

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(5, 2))
pm.plot_posterior(trace, color='skyblue', round_to=2, ax=ax);

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
plt.hist(obs, 
         bins=100, density=True, histtype='stepfilled', facecolor='lightgray', edgecolor='k',
         label='observations');

for mu, sigma in zip(trace['mu'][::100], trace['sd'][::100]):
    pdf = normal_pdf_truncated_v2(x_norm, mu, sigma, upper)
    plt.plot(x_norm, pdf, '-r', alpha=0.01)

plt.plot(np.nan, np.nan, '-r', label='posterior')
plt.legend();